In [ ]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir


def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.0) and (pt2[2] > 0.0):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame


# def visualize(im, detections, poses3d, poses2d, edges, frame_count, output_2d_dir="plots_2d", output_3d_dir="plots_3d"):
#     """Visualize 2D and 3D poses with bounding boxes and save to separate folders."""
#     # Create output directories if they don't exist
#     os.makedirs(output_2d_dir, exist_ok=True)
#     os.makedirs(output_3d_dir, exist_ok=True)

#     # Create figure for 2D plot
#     fig_2d = plt.figure(figsize=(5, 5.2))
#     image_ax = fig_2d.add_subplot(1, 1, 1)
#     image_ax.imshow(im)
#     # for x, y, w, h in detections[:, :4]:
#     #     image_ax.add_patch(Rectangle((x, y), w, h, fill=False))

#     for pose2d in poses2d:
#         for i_start, i_end in edges:
#             image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2)
#         image_ax.scatter(*pose2d.T, s=2)

#     # Save 2D plot
#     plt.tight_layout()
#     plt.savefig(os.path.join(output_2d_dir, f"frame_{frame_count}.png"), dpi=300, bbox_inches='tight')
#     plt.close(fig_2d)

#     # Create figure for 3D plot
#     fig_3d = plt.figure(figsize=(5, 5.2))
#     pose_ax = fig_3d.add_subplot(1, 1, 1, projection='3d')
#     pose_ax.view_init(5, -85)
#     pose_ax.set_xlim3d(-1500, 1500)
#     pose_ax.set_zlim3d(-1500, 1500)
#     pose_ax.set_ylim3d(0, 3000)

#     poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1]
    
#     for pose3d in poses3d:
#         for i_start, i_end in edges:
#             pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2)
#         pose_ax.scatter(*pose3d.T, s=2)

#     # Save 3D plot
#     plt.tight_layout()
#     plt.savefig(os.path.join(output_3d_dir, f"frame_{frame_count}.png"), dpi=300, bbox_inches='tight')
#     plt.close(fig_3d)
import os
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def visualize(im, detections, poses3d, poses2d, edges, frame_count, output_2d_dir="plots_2d", output_3d_dir="plots_3d"):
    """Visualize 2D and 3D poses with bounding boxes and save to separate folders."""
    # Create output directories if they don't exist
    os.makedirs(output_2d_dir, exist_ok=True)
    os.makedirs(output_3d_dir, exist_ok=True)

    # Create figure for 2D plot
    fig_2d = plt.figure(figsize=(5, 5.2))
    image_ax = fig_2d.add_subplot(1, 1, 1)
    image_ax.imshow(im)
    # for x, y, w, h in detections[:, :4]:
    #     image_ax.add_patch(Rectangle((x, y), w, h, fill=False))

    for pose2d in poses2d:
        for i_start, i_end in edges:
            image_ax.plot(*zip(pose2d[i_start], pose2d[i_end]), marker='o', markersize=2, linewidth=2)  # Increased linewidth to 2
        image_ax.scatter(*pose2d.T, s=2)

    # Save 2D plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_2d_dir, f"frame_{frame_count}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig_2d)

    # Create figure for 3D plot
    fig_3d = plt.figure(figsize=(5, 5.2))
    pose_ax = fig_3d.add_subplot(1, 1, 1, projection='3d')
    pose_ax.view_init(5, -85)
    pose_ax.set_xlim3d(-1500, 1500)
    pose_ax.set_zlim3d(-1500, 1500)
    pose_ax.set_ylim3d(0, 3000)

    poses3d[..., 1], poses3d[..., 2] = poses3d[..., 2], -poses3d[..., 1]
    
    for pose3d in poses3d:
        for i_start, i_end in edges:
            pose_ax.plot(*zip(pose3d[i_start], pose3d[i_end]), marker='o', markersize=2, linewidth=2)  # Increased linewidth to 2
        pose_ax.scatter(*pose3d.T, s=2)

    # Save 3D plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_3d_dir, f"frame_{frame_count}.png"), dpi=300, bbox_inches='tight')
    plt.close(fig_3d)

def process_video(video_path, model, skeleton_edges, output_video="output_video3d.mp4", frame_interval=1, output_excel="2dand3dposedatawithlabels.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, save 2D and 3D plots, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()
                boxes = pred.get('boxes', tf.zeros((keypoints_2d.shape[0], 4))).numpy()  # Get bounding boxes if available

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Save 2D and 3D plots
                    visualize(frame_rgb_resized, boxes, keypoints_3d, keypoints_2d[:, :, :2], skeleton_edges, frame_count)

                    # Iterate over detected poses (persons)
                    for i in range(keypoints_2d.shape[0]):
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            # Draw skeleton
                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Add person label (e.g., "Person 1", "Person 2", etc.)
                            person_label = f"Person {i + 1}"  # Person ID starts from 1
                            label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head
                            cv2.putText(frame, person_label, label_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")


def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (mm)': stride_length_left_2d,
        'Stride Length Right (mm)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (mm/s)': gait_speed_left_2d,
        'Gait Speed Right (mm/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }


def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (mm)': stride_length_left_3d,
        'Stride Length Right (mm)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (mm/s)': gait_speed_left_3d,
        'Gait Speed Right (mm/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }

def plot_ankle_z_data(df, output_dir="plots_ankle_z"):
    """Plot Z-direction data for left and right ankles for each person."""
    # Create output directory for ankle Z plots
    os.makedirs(output_dir, exist_ok=True)
    
    # Joint map for ankle indices
    joint_map = {
        'left_ankle': 7,
        'right_ankle': 8
    }
    
    # Get unique person IDs
    unique_persons = df['Person'].unique()
    
    for person_id in unique_persons:
        # Filter data for this person
        df_person = df[df['Person'] == person_id]
        
        # Extract left and right ankle Z-coordinates
        left_ankle_z = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'Z_3D']].sort_values('Frame')
        right_ankle_z = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'Z_3D']].sort_values('Frame')
        
        # Skip if no data for ankles
        if left_ankle_z.empty and right_ankle_z.empty:
            print(f"No ankle data for Person {person_id}")
            continue
        
        # Create plot
        plt.figure(figsize=(10, 6))
        
        # Plot left ankle Z-coordinate
        if not left_ankle_z.empty:
            plt.plot(left_ankle_z['Frame'], left_ankle_z['Z_3D'], label=f'Person {person_id} Left Ankle', color='blue', linestyle='-')
        
        # Plot right ankle Z-coordinate
        if not right_ankle_z.empty:
            plt.plot(right_ankle_z['Frame'], right_ankle_z['Z_3D'], label=f'Person {person_id} Right Ankle', color='red', linestyle='--')
        
        # Customize plot
        plt.xlabel('Frame')
        plt.ylabel('Z-Coordinate (mm)')
        plt.title(f'Ankle Z-Coordinate Movement for Person {person_id}')
        plt.legend()
        plt.grid(True)
        
        # Save plot
        plt.savefig(os.path.join(output_dir, f'person_{person_id}_ankle_z.png'), dpi=300, bbox_inches='tight')
        plt.close()
        
        print(f"Saved ankle Z plot for Person {person_id} to {os.path.join(output_dir, f'person_{person_id}_ankle_z.png')}")
def main():
    # Paths
    zip_path = r"C:\\Users\\akhileshsing2024\\Downloads\\metrabs_eff2l_y4_360.zip"
    video_path = r"C:\Users\akhileshsing2024\Desktop\Curve_walking_No_task\IMG_0597.MOV"
    output_excel = "2dand3dposedatawithlabels.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video (this generates the initial Excel file with 2D in pixels, 3D in mm)
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)

    # Step 1: Compute pixel-to-mm conversion factor using hip-to-ankle distance
    unique_persons = df['Person'].unique()
    pixel_to_mm_factors = []

    for person_id in unique_persons:
        df_person = df[df['Person'] == person_id]
        
        # Extract hip and ankle coordinates for all frames
        left_hip_2d = df_person[df_person['Joint'] == 1][['Frame', 'X_2D', 'Y_2D']]
        left_ankle_2d = df_person[df_person['Joint'] == 7][['Frame', 'X_2D', 'Y_2D']]
        left_hip_3d = df_person[df_person['Joint'] == 1][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]
        left_ankle_3d = df_person[df_person['Joint'] == 7][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]

        # Merge on Frame to get corresponding hip and ankle coordinates
        hip_ankle_2d = left_hip_2d.merge(left_ankle_2d, on='Frame', suffixes=('_hip', '_ankle'))
        hip_ankle_3d = left_hip_3d.merge(left_ankle_3d, on='Frame', suffixes=('_hip', '_ankle'))

        # Compute distances for each frame
        distances_2d = []
        distances_3d = []
        for idx in range(len(hip_ankle_2d)):
            # 2D distance (pixels)
            hip_2d = hip_ankle_2d.iloc[idx][['X_2D_hip', 'Y_2D_hip']].values
            ankle_2d = hip_ankle_2d.iloc[idx][['X_2D_ankle', 'Y_2D_ankle']].values
            dist_2d = distance.euclidean(hip_2d, ankle_2d)

            # 3D distance (mm)
            hip_3d = hip_ankle_3d.iloc[idx][['X_3D_hip', 'Y_3D_hip', 'Z_3D_hip']].values
            ankle_3d = hip_ankle_3d.iloc[idx][['X_3D_ankle', 'Y_3D_ankle', 'Z_3D_ankle']].values
            dist_3d = distance.euclidean(hip_3d, ankle_3d)

            if dist_2d > 0:  # Avoid division by zero
                pixel_to_mm = dist_3d / dist_2d
                pixel_to_mm_factors.append(pixel_to_mm)
                distances_2d.append(dist_2d)
                distances_3d.append(dist_3d)

    # Compute the average pixel-to-mm factor
    if pixel_to_mm_factors:
        pixel_to_mm = np.mean(pixel_to_mm_factors)
        print(f"Computed pixel-to-mm conversion factor: {pixel_to_mm:.4f} mm/pixel")
        print(f"Average 3D hip-to-ankle distance: {np.mean(distances_3d):.2f} mm")
        print(f"Average 2D hip-to-ankle distance: {np.mean(distances_2d):.2f} pixels")
    else:
        # Fallback to anthropometric average if no valid distances are computed
        pixel_to_mm = 900 / 500  # Assume 900 mm leg length, 500 pixels in 2D (rough estimate)
        print(f"No valid distances computed. Using fallback pixel-to-mm factor: {pixel_to_mm:.4f} mm/pixel")

    # Step 2: Convert 2D coordinates to millimeters
    df['X_2D'] = df['X_2D'] * pixel_to_mm
    df['Y_2D'] = df['Y_2D'] * pixel_to_mm

    # Step 3: Save the updated Excel file with 2D and 3D coordinates in millimeters
    updated_excel = "2dand3dposedatawithlabels_in_mm.xlsx"
    df.to_excel(updated_excel, index=False)
    print(f"Updated pose data with 2D coordinates in millimeters saved as {updated_excel}")

    plot_ankle_z_data(df)

    # Step 4: Calculate gait metrics
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    print(f"Detected {len(unique_persons)} persons in the video.")

    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics (now in millimeters)
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics (already in millimeters)
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all detected persons saved to 'gait_metrics_per_person.xlsx'")


if __name__ == '__main__':
    main()

Model loaded successfully!


KeyboardInterrupt: 